In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn.preprocessing import LabelEncoder
import pickle

In [2]:
df=pd.read_csv(r"autos.csv",header=0,sep=',',encoding='Latin1',)

In [3]:
df[df.seller != 'gewerblich']
df=df.drop( 'seller', 1)
df[df.offerType != 'Gesuch']
df=df.drop( 'offerType', 1)

C:\Users\GANESH\AppData\Local\Temp\ipykernel_32536\3958108311.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df=df.drop( 'seller', 1)
C:\Users\GANESH\AppData\Local\Temp\ipykernel_32536\3958108311.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df=df.drop( 'offerType', 1)


In [4]:
df = df[ (df.powerPS > 50) & (df.powerPS < 900) ]
df = df[ (df.yearOfRegistration >= 1950) & (df.yearOfRegistration < 2017)]

In [5]:
df.drop(['name','abtest','dateCrawled','nrOfPictures','lastSeen','postalCode','dateCreated'],axis='columns',inplace=True)

In [6]:
new_df = df.copy()
new_df = new_df.drop_duplicates(['price','vehicleType','yearOfRegistration','gearbox','powerPS','model','kilometer','monthOfRegistration','fuelType','notRepairedDamage'])

In [7]:
new_df.gearbox.replace(('manuell','automatik'),('manual','automatic'),inplace=True)
new_df.fuelType.replace(('benzin','andere','elektro'),('petrol','others','electirc'),inplace=True)
new_df.vehicleType.replace(('kleinwagen','cabrio','kombi','andere'),('small car','convertible','combination','others'),inplace=True)
new_df.notRepairedDamage.replace(('ja','nein'),('Yes','No'),inplace=True)

In [8]:
new_df = new_df[(new_df.price >= 100) & (new_df.price <= 150000)]

In [9]:
new_df['notRepairedDamage'].fillna(value='not-declared',inplace=True)
new_df['fuelType'].fillna(value='not-declared',inplace=True)
new_df['gearbox'].fillna(value='not-declared',inplace=True)
new_df['vehicleType'].fillna(value='not-declared',inplace=True)
new_df['model'].fillna(value='not-declared',inplace=True)

In [10]:
new_df.to_csv("autos_preprocessed.csv")

#label encoding the categorical data
labels = ['gearbox','notRepairedDamage','model','brand','fuelType','vehicleType']

In [11]:
mapper = {}
for i in labels:
	mapper[i] = LabelEncoder()
	mapper[i].fit(new_df[i])
	tr = mapper[i].transform(new_df[i])
	np.save(str('classes'+i+'.npy'),mapper[i].classes_)
	print(i,";",mapper[i])
	new_df.loc[:,i+'_labels'] = pd.Series(tr,index = new_df.index)
labeled = new_df[ [ 'price' , 'yearOfRegistration','powerPS','kilometer','monthOfRegistration'] + [x+"_labels" for x in labels]]

gearbox ; LabelEncoder()
notRepairedDamage ; LabelEncoder()
model ; LabelEncoder()
brand ; LabelEncoder()
fuelType ; LabelEncoder()
vehicleType ; LabelEncoder()


In [12]:
print(labeled.columns)

Index(['price', 'yearOfRegistration', 'powerPS', 'kilometer',
       'monthOfRegistration', 'gearbox_labels', 'notRepairedDamage_labels',
       'model_labels', 'brand_labels', 'fuelType_labels',
       'vehicleType_labels'],
      dtype='object')


In [13]:
print(labeled.columns)

Index(['price', 'yearOfRegistration', 'powerPS', 'kilometer',
       'monthOfRegistration', 'gearbox_labels', 'notRepairedDamage_labels',
       'model_labels', 'brand_labels', 'fuelType_labels',
       'vehicleType_labels'],
      dtype='object')


In [14]:
Y = labeled.iloc[:,0].values
X = labeled.iloc[:,1:].values

In [15]:
Y = Y.reshape(-1,1)
from sklearn.model_selection import cross_val_score , train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.3,random_state=3)

In [16]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
regressor = RandomForestRegressor(n_estimators = 1000,max_depth=10,random_state=34)
regressor.fit(X_train,np.ravel(Y_train,order='C'))

RandomForestRegressor(max_depth=10, n_estimators=1000, random_state=34)

In [17]:
y_pred = regressor.predict(X_test)
print(r2_score(Y_test,y_pred))

0.834527626497731


In [18]:
filename = 'resale_model.sav'
pickle.dump(regressor,open(filename,'wb'))

In [19]:
!pip install -U ibm-watson-machine-learning

Defaulting to user installation because normal site-packages is not writeable


In [20]:
from ibm_watson_machine_learning import APIClient

In [21]:
wml_credentials = {
    "apikey":"y9fOFST6bMUm-ugNLk5iQoRH9dK4auxmxI6Wj61jiF5R",
    "url":"https://us-south.ml.cloud.ibm.com"
}

In [22]:
client = APIClient(wml_credentials)
client.spaces.list()

Note: 'limit' is not provided. Only first 50 records will be displayed if the number of records exceed 50
------------------------------------  -----  ------------------------
ID                                    NAME   CREATED
7dae8252-921f-4da4-9086-60d2b0466f5e  Model  2022-11-17T06:23:23.016Z
------------------------------------  -----  ------------------------


In [23]:
SPACE_ID="7dae8252-921f-4da4-9086-60d2b0466f5e"

In [24]:
client.set.default_space(SPACE_ID)

'SUCCESS'

In [25]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [28]:
software_spec_uid=client.software_specifications.get_uid_by_name("runtime-22.1-py3.9")
software_spec_uid

'12b83a17-24d8-5082-900f-0ab31fbfd3cb'

In [29]:
 model_details=client.repository.store_model(model=regressor,meta_props={
 client.repository.ModelMetaNames.NAME:"CRV",
 client.repository.ModelMetaNames.TYPE:"scikit-learn_1.0",
 client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid}
                                            
)
model_id=client.repository.get_model_uid(model_details)

This method is deprecated, please use get_model_id()


C:\Users\GANESH\AppData\Roaming\Python\Python39\site-packages\ibm_watson_machine_learning\repository.py:1453: UserWarning: This method is deprecated, please use get_model_id()
  warn("This method is deprecated, please use get_model_id()")


In [30]:
model_id

'0e5be457-c8ac-4d67-b10d-1dbb9311497f'

In [31]:
Y_pred=regressor.predict(X_test)

In [34]:
y_pred

array([9937.11993609, 1157.33268937, 2739.06719477, ..., 2124.91777611,
       2955.66758818, 1207.44132337])